In [1]:
# Import necessary libraries
import faiss
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import streamlit as st

# Load the job content data
job_content_df = pd.read_csv(r"C:\Users\terbe\Downloads\job_content_fixed.csv")

job_content_df




,Unnamed: 0,Class Code,Class Title,Prob Period,Spec Effective Date,Exam Effective Date,Time Limit,Typing Test,Occ Line,Work Area,Last Action,Test Type,Knowledge Test,Job Duties,Count,KSA,Levels Of Work,MAQ,Combined_String
0,0,4755,Academic Contract Specialist,6,2022-08-31,2022-08-31,0.0,No,Clerical,Campus,Revised,E-Test,No,Employees at this level perform highly special...,1,Extensive knowledge of general office practice...,maintains a complex data system that includes ...,1. High school diploma or equivalent. 2. Any o...,Job Duties: Employees at this level perform hi...
1,1,4763,Academic Contract Supervisor,6,2022-08-31,2022-08-31,0.0,No,Clerical,Campus,Revised,E-Test,No,Employees at this level are responsible for su...,1,"Knowledge of data collection methods, statisti...",supervises a group of Academic Contract Specia...,1. High school diploma or equivalent. 2. Any o...,Job Duties: Employees at this level are respon...
2,2,10,Accountant I,6,2022-07-01,2022-07-01,0.0,No,Professional,Accounting,Revised,E-Test,No,Employees in positions allocated to this level...,1,Problem-solving (Ability to tell when somethin...,"reconciles reports and bank statements, noting...",1. Any one or any combination of the following...,Job Duties: Employees in positions allocated t...
3,3,11,Accountant II,6,2022-07-01,2022-07-01,0.0,No,Professional,Accounting,Revised,E-Test,No,Employees in positions allocated to this level...,1,Problem-solving (Ability to tell when somethin...,prepares annual financial schedules for state ...,1. Any one or any combination of the following...,Job Duties: Employees in positions allocated t...
4,4,12,Accountant III,12,2022-07-01,2022-07-01,0.0,No,Professional,Accounting,Revised,E-Test,No,Employees in positions allocated to this level...,1,Problem-solving (Ability to tell when somethin...,directs the day-to-day operation of a small ac...,1. Bachelor’s degree with a major in accountan...,Job Duties: Employees in positions allocated t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,865,4191,Waiter/Waitress,6,2022-10-15,2022-10-15,0.0,No,Food Services,Food Serving,Revised,E-Test,No,Under direct supervision from a designated sup...,1,Knowledge of correct methods of setting tables...,sets up tables with clean linens and silverwar...,1. High school diploma or equivalent. nan,Job Duties: Under direct supervision from a de...
866,866,1836,Water Station Foreman,6,1987-08-21,2022-10-15,0.0,No,"Power, Heat, and Light Services","Heat, Light, and Power Services",Revised,E-Test,No,Employees at this level supervise and direct a...,1,Knowledge of chemical tests pertaining to the ...,directs and is responsible for the operation o...,1. High school diploma or equivalent. 2. Four ...,Job Duties: Employees at this level supervise ...
867,867,3003,Water Station Operator,6,2007-11-26,2022-10-15,0.0,No,"Power, Heat, and Light Services","Heat, Light, and Power Services",Revised,E-Test,No,Employees at this level assist in operating an...,1,Ability to perform chemical tests pertaining t...,"inspects, tests, and chemically treats water i...",1. High school diploma or equivalent. 2. Certi...,Job Duties: Employees at this level assist in ...
868,868,1969,Water Station Sub-Foreman,6,1987-08-21,2022-10-15,0.0,No,"Power, Heat, and Light Services","Heat, Light, and Power Services",Revised,E-Test,No,Employees at this level are experienced operat...,1,Knowledge of chemical tests pertaining to the ...,assists in the supervision of Water Station Op...,1. High school diploma or equivalent. 2. Three...,Job Duties: Employees at this level are experi...


In [ ]:
# Preprocess the 'Combine_String' column, filling NaN values with empty strings
combine_strings = job_content_df['Combined_String'].fillna('')

# Load a pre-trained BERT-based model for embeddings
bert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Convert job summaries to BERT embeddings
bert_embeddings = bert_model.encode(combine_strings.tolist(), convert_to_tensor=True)

# Set up FAISS index for BERT embeddings
dimension = bert_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)

# Add the BERT embeddings to the FAISS index
faiss_index.add(bert_embeddings.cpu().numpy())

# Save BERT embeddings to a local file
np.save(r'C:\Users\terbe\Desktop\Job Classification - Wrap Up\bert_embeddingsx.npy', bert_embeddings.cpu().numpy())

# Save FAISS index to a local file
faiss.write_index(faiss_index, r'C:\Users\terbe\Desktop\Job Classification - Wrap Up\faiss_indexx.index')

In [7]:
check_this = job_content_df[job_content_df["Class Code"] == 5030]
check_this

,Unnamed: 0,Class Code,Class Title,Levels Of Work,MAQ,KSA,Job Duties,Promotional Line,Prob Period,Spec Effective Date,Exam Effective Date,Time Limit,Typing Test,Occ Line,Work Area,Last Action,Test Type,Knowledge Test,Combine_String
865,865,5030,Information Technology Manager/Administrative ...,Technological Expertise. Budget/Resources. Str...,1. Any one or any combination totaling 5 years...,Knowledge of the systems and operations used w...,These positions are responsible for managing s...,NaN,12,2023-04-01,2023-04-01,0.0,No,Professional,Special Group,Revised,E-Test,No,Information Technology Manager/Administrative ...


In [10]:
check_this = job_content_df[job_content_df["Class Code"] == 5030]

# Extracting and printing each value as a plain string
print("Levels of Work: " + str(check_this["Levels Of Work"].iloc[0]))
print(" ")
print("MAQ: " + str(check_this["MAQ"].iloc[0]))
print(" ")
print("KSA: " + str(check_this["KSA"].iloc[0]))
print(" ")
print("Job Duties: " + str(check_this["Job Duties"].iloc[0]))
print(" ")
print("Combine_String: " + str(check_this["Combine_String"].iloc[0]))


Levels of Work: Technological Expertise. Budget/Resources. Strategic Planning. Supervision. Customer Service. Cybersecurity. Project Management. Additional Related Duties
 
MAQ: 1. Any one or any combination totaling 5 years (60 months) from the following categories:. A. College coursework which includes Information Technology (IT), IT Management, Programming, IT Systems, or a closely related discipline, as measured by the following conversion table or its proportional equivalent:. 30 semester hours equals one (1) year (12 months). Associate’s Degree (60 semester hours) equals eighteen months (18 months). 90 semester hours equals two (2) years (24 months). Bachelor’s Degree (120 semester hours) equals three (3) years (36 months). B. Work experience in IT-related functions such as hardware/software support, programming, network design, network engineering, IT systems integration, or closely related field, which includes supervisory or lead worker capacity.. 2. Based on position requirem

In [ ]:

# Function to retrieve similar jobs using BERT embeddings
def retrieve_similar_jobs_bert(new_job_summary, faiss_index, bert_model, top_k=5):
    # Convert new job summary to BERT embedding
    new_embedding = bert_model.encode([new_job_summary], convert_to_tensor=True)
    
    # Use FAISS to find the top k most similar jobs
    _, top_k_indices = faiss_index.search(new_embedding.cpu().numpy(), top_k)
    
    # Retrieve corresponding job descriptions
    similar_jobs = job_content_df.iloc[top_k_indices[0]]
    return similar_jobs

# Streamlit App
st.title("Job Search App")

# User Input for Job Query
user_query = st.text_input("Enter a job query (e.g., 'I need an accountant with 4 years of experience'): ")

# Allow user to select the number of returned job results (k)
top_k = st.number_input("Enter number of job results to return", min_value=1, max_value=20, value=5)

# Retrieve jobs based on the user query
if user_query:
    # Display possible jobs retrieved using BERT and FAISS
    similar_jobs_bert = retrieve_similar_jobs_bert(user_query, faiss_index, bert_model, top_k=top_k)
    
    # Display a selection box for the user to pick a job title
    selected_job_title = st.selectbox("Select a job title", similar_jobs_bert['Class Title'].tolist())
    
    # Display full job posting when a job title is selected
    if selected_job_title:
        selected_job = similar_jobs_bert[similar_jobs_bert['Class Title'] == selected_job_title].iloc[0]
        
        # Display job posting details
        st.write(f"### {selected_job['Class Title']}")
        st.write(f"**Job Duties:** {selected_job['Job Duties']}")
        st.write(f"**Minimum Qualifications:** {selected_job['MAQ']}")
        st.write(f"**Levels of Work:** {selected_job['Levels Of Work']}")
        st.write(f"**Knowledge, Skills, Abilities (KSA):** {selected_job['KSA']}")